In [12]:
import pandas as pd 
import numpy as np 
import re 
import os
import sys
import urllib.request
import json
from tabulate import tabulate

**<제품명에서 제거된 것 + 이에 사용한 정규표현식 ('|'로 구분)>**

**1. 괄호를 포함 혹은 포함하지 않은 결제관련 정보(무이자, 일시불, 이자 등)**

    \(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?

**2. 제품명 맨 앞, 맨 끝에 붙어있는 괄호들 (40인치) (특상품) (초특가) 등은 제품 카테고리와 무방하다 생각하여 삭제**

    \(.{1,10}\)$|^\(.?\)

**3. ~패키지, ~종, 세트, n+n, 더블팩, 싱글팩 등 제품 수량단위와 관련한 것**

    \s?패키지\s?|[0-9]+종.*|세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}

# Preprocess (민수 ipynb 참고)

In [55]:
t = pd.read_excel('C:/Users/SHJ/Documents/GIS/Bigcon2020/data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', skiprows=1)
print(t.isnull().sum(),'\n')
t=t[t['상품군']!='무형']
print(t.isnull().sum(),'\n')
t[t['취급액'].isnull()]
t.취급액 = t.취급액.fillna(0)
print(t.isnull().sum())
t

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액       2930
dtype: int64 

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액       1993
dtype: int64 

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액          0
dtype: int64


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
38301,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
38302,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


In [56]:
performance = pd.read_excel('C:/Users/SHJ/Documents/GIS/Bigcon2020/data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', skiprows=1)
ratings = pd.read_excel('C:/Users/SHJ/Documents/GIS/Bigcon2020/data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx', skiprows=1)
performance = performance[performance['상품군']!='무형']
performance['취급액'] = performance['취급액'].fillna(0)
performance = performance.fillna(method='pad')
performance.drop(['마더코드','상품코드'], axis=1, inplace=True)
performance['방송일시_yymmdd'] = performance['방송일시'].apply(lambda x: str(x)[:10])
performance['방송일시_hhmmss'] = performance['방송일시'].apply(lambda x: str(x)[11:])

performance['seconds'] = performance['방송일시_hhmmss'].apply(lambda x: int(x[:2])*60 + int(x[3:5]))


performance.head()


,방송일시,노출(분),상품명,상품군,판매단가,취급액,방송일시_yymmdd,방송일시_hhmmss,seconds
0,2019-01-01 06:00:00,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,06:00:00,360
1,2019-01-01 06:00:00,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,06:00:00,360
2,2019-01-01 06:20:00,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,06:20:00,380
3,2019-01-01 06:20:00,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,06:20:00,380
4,2019-01-01 06:40:00,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,06:40:00,400


In [57]:
ratings = ratings.set_index('시간대')
ratings.mean(axis=1)
ratings.drop('월화수목금토일02:00-01:59', axis=0, inplace=True)

mean = ratings.mean(axis=1).values
ratings['mean'] = mean
ratings = ratings.reset_index()
ratings = ratings[['시간대','mean']]
ratings['시간대'] = ratings['시간대'].astype('datetime64[ns]')
ratings.set_index('시간대', inplace=True)

# 5단위로 시간대별 평균 집계
tmp = ratings[['mean']].resample('5T').mean().reset_index()
ratings = ratings.reset_index()
ratings = pd.merge(ratings, tmp, on='시간대', how='left')
ratings.head()

ratings.drop('mean_x', axis = 1, inplace= True)
ratings.rename({'mean_y':'ratings_mean'}, axis=1, inplace=True)
ratings['시간대'] = ratings['시간대'].astype(str)
ratings.head()

# 시간대에서 초단위 삭제
ratings['시간대'] = ratings['시간대'].apply(lambda x: x[-8:-3])
# 시분만 사용한 컬럼 생성
performance['방송일시_hhmm'] = performance['방송일시_hhmmss'].apply(lambda x: x[:-3])

performance = pd.merge(performance, ratings, left_on = '방송일시_hhmm', right_on = '시간대', how='left')
del performance['시간대']

performance.head()

,방송일시,노출(분),상품명,상품군,판매단가,취급액,방송일시_yymmdd,방송일시_hhmmss,seconds,방송일시_hhmm,ratings_mean
0,2019-01-01 06:00:00,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,06:00:00,360,06:00,0.002255
1,2019-01-01 06:00:00,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,06:00:00,360,06:00,0.002255
2,2019-01-01 06:20:00,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,06:20:00,380,06:20,0.002210
3,2019-01-01 06:20:00,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,06:20:00,380,06:20,0.002210
4,2019-01-01 06:40:00,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,06:40:00,400,06:40,0.001997


# 제품명 전처리

In [176]:
# 상품명 중복 제거
uniq_pname = performance['상품명'].unique()
print('\n<# of Unique pnameuct Name> : \n\t{}'.format(len(uniq_pname)))
np.set_printoptions(threshold=sys.maxsize)

# 추후 조합을 위해 전처리 이전 제품명과 이후 제품명을 저장합니다.
p_name = pd.DataFrame(data=uniq_pname, columns=['Before Process'])

# 제품명에서 불필요한 요소 전처리
ap=[]
re_stop = re.compile("""\([가-힣]{1,5}\+[가-힣]{1,5}\)|\(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?|무료체험|
|포함|국내[가-힣]+\s|무료설치|\s?신제품\s?|\s?패키지\s?|[0-9]+종|풀코디|set|SET|풀세트|[0-9]+세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}|[0-9]{1,2}인용|[0-9]{1,2}박스|[0-9\.]{1,4}미터|[0-9\.]{1,4}kg|[0-9\.]{1,4}[kKgG]|[0-9]{1,3}[벌롤종단구대P개통병포미봉팩장gL매]|
|\s[0-9\.]{1,4}[Mm]|\(.{1,10}\)$|^\(.?\)|^[0-9]{2,4}\s|\s+[0-9]{2,4}\s|[0-9]{2,4}년\s|[0-9]{2,4}년형\s|시즌[0-9]|[0-9]{1,3}\%|\(.{1,20}\)$|기본형|고급형|오리지널|[대중소大中小]형|.{1,10}by|,|\s?총[0-9\s]|^[가-힣a-zA-Z]{3,3}의|
|S\/S|F\/W|f\/w|s\/s|[가-힣]+형\s|[슈퍼]{0,2}싱글|\s[SQK퀸킹]{1,2}\s|[SQK퀸킹]{1,2}$|[가-힣]{0,2}사이즈""")

remnants = re.compile('[",g\+lL-]\s|ml|_|\sx|[\[\]\］!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]|\s종$|\s[0-9]+\s|\s{2,}')

for i in p_name['Before Process']:
    tmp = re_stop.sub(' ',i)
    tmp = tmp.strip()
    tmp = remnants.sub(' ',tmp)
    ap.append(tmp.strip())

# 전처리된 제품명 중복제거
p_name['After Process'] = ap
p_name.to_html('tmp.html')

ap = list(set(ap))

p_name


<# of Unique pnameuct Name> : 
	1692


,Before Process,After Process
0,테이트 남성 셀린니트3종,테이트 남성 셀린니트
1,테이트 여성 셀린니트3종,테이트 여성 셀린니트
2,오모떼 레이스 파운데이션 브라,오모떼 레이스 파운데이션 브라
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,PAT 남성 소프트 기모 릴렉스팬츠
4,보코 리버시블 무스탕,보코 리버시블 무스탕
...,...,...
1687,조방할매 낙곱새 6팩세트,조방할매 낙곱새 세트
1688,파격가 니봇 무선 진공 물걸레 로봇청소기 +뉴헤어드라이어기,파격가 니봇 무선 진공 물걸레 로봇청소기 뉴헤어드라이어기
1689,램프쿡 자동회전냄비,램프쿡 자동회전냄비
1690,벨라홈 스마트 멀티포트 1+1 세트,벨라홈 스마트 멀티포트 세트


In [120]:
print(ap)

['프라다 투브릿지 선글라스', '사운드룩 LP턴테이블 오디오 SLT 7080BT', '투홀릭 풍기인견 드로즈', '삼성 블루스카이 공기청정기 AX80R7580WDD', 'PAT 남성 캐주얼 셔츠', '손질 통오징어', '임페리얼 여성 패딩코트세트', '프라다 자카드 카메라 크로스백', '마르엘라로사티 쿨썸머 데님', '발레리 라이크라 풀커버 컴포트핏 브라', '이동수골프 남성 원더 플렉스 본딩팬츠', '오모떼 리얼스킨 쉐이핑 브라팬티', '헤스티지 엘레나 라쿤양가죽 콤비 롱코트', '삼성노트북 북5NT550EBE K34W', '보코 쿨썸머 메쉬 앙상블', '하늘내린 용대리황태', 'NNF 남성 터틀넥', '키친아트 스마트뷰 스텐밀폐용기', '루이띠에  인피니티 러브 귀걸이', '구찌 씨마 베이직 양면 울 머플러', '에펨 여성 글로스 샌들', '까사마루 접이식 테이블 세트', '스팀큐다리미', '팜스프링스 남성 패딩팬츠', '코튼데이 유기농 순면 마스크 KF94', '월드컵 여성러닝화 트레킹화', '서장훈의 락앤락 칼도마 살균블럭', '목우촌 1등급 오리로스', '칼리베이직 여성백 세트', '리복 스피드윅 드로즈', '도스문도스 카이만 엠보 소가죽 토트백', '중외신약 빠르카 통증패치', 'K SWISS  남성 패딩 트랙수트', '월드컵 남성 러닝화 슬리퍼', '삼성 UHD TV UN75NU7050F', '단하루 종근당건강 프로메가 오메가3 파워 비타민D', '디즈니 여성 트랙수트', '크리스티나앤코 썸머플리츠세트', '붙이는 따스림 실크 단열벽지 미터', '뉴트리원 다이어트 구미', '코치 뉴 미아 숄더백', '도스문도스 카이만 엠보 소가죽 체인 숄더백', '삭루츠 호보백', '일월 순면 온열매트', '에트로 프로푸미 숄더   크로스백', '뉴욕맘 항균 수세미', '루이띠에  그레이스 골드 반지', '레코바 여성 어반', '루이띠에 순금 신디스틱 팔찌  골드바', '아가타 소가죽 데이쿠션로퍼', '파격가 굿프렌드 스트레칭 안마 매트 GOOD A7', '알비

In [125]:
len(new_uprod)
print(len(ap))
print("DF 전체 상품 수 : {}\n유니크 상품 수 : {}\n전처리후 유니크 상품 수 : {}".format(len(df['상품명']), len(u_prod),len(ap)))

1266
DF 전체 상품 수 : 37372
유니크 상품 수 : 1692
전처리후 유니크 상품 수 : 1266


In [110]:
p_name

,Before Process,After Process
0,테이트 남성 셀린니트3종,테이트 남성 셀린니트
1,테이트 여성 셀린니트3종,테이트 여성 셀린니트
2,오모떼 레이스 파운데이션 브라,오모떼 레이스 파운데이션 브라
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,PAT 남성 소프트 기모 릴렉스팬츠
4,보코 리버시블 무스탕,보코 리버시블 무스탕
...,...,...
1687,조방할매 낙곱새 6팩세트,조방할매 낙곱새 세트
1688,파격가 니봇 무선 진공 물걸레 로봇청소기 +뉴헤어드라이어기,파격가 니봇 무선 진공 물걸레 로봇청소기 뉴헤어드라이어기
1689,램프쿡 자동회전냄비,램프쿡 자동회전냄비
1690,벨라홈 스마트 멀티포트 1+1 세트,벨라홈 스마트 멀티포트 세트


**혜인**

ID: OpYVfNnlgdKhnWHmxlxa

PW: acYb7du6_E


**왕수**

ID: tdrEWZrN1ZQp9u0yqy3e

PW: 59Bl3ltKdX

In [193]:
def naver_shop(keyword):
    client_id = "DWNxDn7nWmw_X5LC74yj"
    client_secret = "MFrEW9wCWI"
    keyword = keyword.strip()
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1" # json 결과
    # url = "https://openapi.naver.com/v1/search/shop.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        jsonString = response_body.decode('utf-8')
        j = json.loads(jsonString)
        if int(j['total']):
            c1 = j["items"][0]['category1']
            c2 = j["items"][0]["category2"]
            c3 = j["items"][0]["category3"]
            c4 = j["items"][0]["category4"]
            keyword2=''
        else :
            p_code = re.compile('[0-9a-zA-Z]+$|[一-龥]+$')
            keyword2 = p_code.sub(' ',keyword)
            keyword2 = keyword2.strip()
            keyword2 = p_code.sub(' ',keyword2)
            keyword2 = ' '.join(keyword2.split()[-2:])
            
            encText = urllib.parse.quote(keyword2)
            url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1"
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            
            if(rescode==200):
                response_body = response.read()
                jsonString = response_body.decode('utf-8')
                j = json.loads(jsonString)
                if int(j['total']):
                    c1 = j["items"][0]['category1']
                    c2 = j["items"][0]["category2"]
                    c3 = j["items"][0]["category3"]
                    c4 = j["items"][0]["category4"]
                else :
                    print(keyword,'->',keyword2)
                    print('>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<')
                    c1, c2, c3, c4 = '', '', '', ''
            else :
                print("Error Code:" + rescode)
    else:
        print("Error Code:" + rescode)
    return [keyword,c1,c2,c3,c4,keyword2]


In [194]:
from tqdm import tqdm_notebook

cat_df = pd.DataFrame(columns = ['pname','cat1','cat2','cat3','cat4','pname2'])

for item in tqdm_notebook(ap):
#     print(item)
    ns = naver_shop(item)
    cat_df=cat_df.append(pd.DataFrame([ns],columns=['pname','cat1','cat2','cat3','cat4','pname2']), ignore_index=True)

cat_df

거창특등급사과 -> 거창특등급사과
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
아가타 컬러풀에디션 -> 아가타 컬러풀에디션
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
자미코코 매트릭스 -> 자미코코 매트릭스
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
메이듀 남성 린넨 블렌디드 슬립온 -> 블렌디드 슬립온
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
스튜디오럭스 후드롱구스다운코트 -> 스튜디오럭스 후드롱구스다운코트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
더커진거창특등급사과 -> 더커진거창특등급사과
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
한샘 뉴퍼스티지R 리클라이너 -> 뉴퍼스티지R 리클라이너
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
도냄  쿠쿠전기밥솥 -> 도냄 쿠쿠전기밥솥
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
바다원  더 그릴 아구채  양태채  쥐포채  소스 -> 쥐포채 소스
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
삼익가구 LED 제니비 침대 -> 제니비 침대
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
삼성5도어냉장고 T9000 RF84R9203S8   시카고커트러리5p세트 -> RF84R9203S8 시카고커트러리5p세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
레이프릴 더블엑스 매직니퍼팬티 -> 더블엑스 매직니퍼팬티
>>>>>>>>>>>>>>>

,pname,cat1,cat2,cat3,cat4,pname2
0,프라다 투브릿지 선글라스,패션잡화,선글라스/안경테,선글라스,,
1,사운드룩 LP턴테이블 오디오 SLT 7080BT,디지털/가전,음향가전,턴테이블,,
2,투홀릭 풍기인견 드로즈,패션의류,남성언더웨어/잠옷,팬티,,
3,삼성 블루스카이 공기청정기 AX80R7580WDD,디지털/가전,계절가전,공기정화기,공기청정기,
4,PAT 남성 캐주얼 셔츠,패션의류,남성의류,셔츠/남방,,
...,...,...,...,...,...,...
1261,레드캠프 남성 썸머 기능성 셔츠,스포츠/레저,등산,등산의류,반팔티셔츠,
1262,유캐슬 마리노 침실가구,가구/인테리어,침실가구,화장대,일반화장대,
1263,트레스패스 여성 다이얼락 방한화 기모 워킹화,패션잡화,남성신발,운동화,워킹화,기모 워킹화
1264,컬럼비아 남성 옴니히트 업 베이스레이어,패션의류,남성의류,트레이닝복,,업 베이스레이어


### 찾지 못한 키워드는


1) 제품코드일 가능성이 높은 알파벳+숫자만으로 이루어진 덩어리 제거

2) 제품 명사일 가능성이 높은 맨 뒷 2어절 탐색




그래도 못찾으면?

1) 형태소 분석기를 통해 띄어쓰기를 다시 하고

2) 브랜드명일 가능성이 높은 맨 앞 noun 제거

3) 다시 탐색



최후의 수단

1) 유사도 기준 참조

In [182]:
cat_df[cat_df['cat1']=='']

,pname,cat1,cat2,cat3,cat4
214,거창특등급사과,,,,
225,아가타 컬러풀에디션,,,,
244,자미코코 매트릭스,,,,
250,무풍클래식 스탠드에어컨AF19R7573WZ,,,,
266,블렌디드 슬립온,,,,
311,스튜디오럭스 후드롱구스다운코트,,,,
314,더커진거창특등급사과,,,,
332,뉴퍼스티지R 리클라이너,,,,
353,도냄 쿠쿠전기밥솥,,,,
375,쥐포채 소스,,,,


In [240]:
kkma = Kkma()
kkma.nouns('자미코코 매트릭스')

['자미', '자미코코', '코', '매트릭스']

In [195]:
from konlpy.tag import Kkma
kkma=Kkma()
def jaccard_sim(doc1, doc2):
    doc1 = kkma.nouns(doc1)
    doc2 = kkma.nouns(doc2)
    u_doc1 = set(doc1)
    u_doc2 = set(doc2)
    return len(u_doc1 & u_doc2) / len(u_doc1 | u_doc2)

In [196]:
cat_df.to_csv('좀더_전처리_카테고리결과.csv')

In [197]:
len(cat_df)
unknown_ratio = len(cat_df[cat_df['cat1']==''])/len(cat_df)*100
print("알아내지 못한 카테고리 비율 : ", round(unknown_ratio,3),"%")

알아내지 못한 카테고리 비율 :  3.239 %


In [201]:
unknown = cat_df[cat_df['cat1']=='']
known = cat_df[cat_df['cat1']!='']
for i in tqdm_notebook(range(len(unknown))):
    best_sim = 0
    print(unknown.iloc[i].pname)
    for j in tqdm_notebook(range(len(known))):
        sim = jaccard_sim(unknown.iloc[i].pname,known.iloc[j].pname)
        if sim > best_sim :
            best_sim = sim
            best_ind = j
            print(best_sim)
    unknown.iloc[i].cat1 = known.iloc[best_ind].cat1
    unknown.iloc[i].cat2 = known.iloc[best_ind].cat2
    unknown.iloc[i].cat3 = known.iloc[best_ind].cat3
    unknown.iloc[i].cat4 = known.iloc[best_ind].cat4
    print(known.iloc[best_ind].cat1, known.iloc[best_ind].cat2,known.iloc[best_ind].cat3,known.iloc[best_ind].cat4)
    
    

거창특등급사과


0.2

식품 농산물 과일 포도
아가타 컬러풀에디션


0.05555555555555555
0.09090909090909091
0.23076923076923078
0.3333333333333333
0.5

화장품/미용 베이스메이크업 파운데이션 쿠션형
자미코코 매트릭스


0.1
0.1111111111111111
0.125
0.16666666666666666
0.21428571428571427
0.23076923076923078
0.8

가구/인테리어 침실가구 매트리스 더블매트리스
메이듀 남성 린넨 블렌디드 슬립온


0.1111111111111111
0.125
0.21428571428571427
0.2727272727272727

패션의류 여성의류 블라우스/셔츠 
스튜디오럭스 후드롱구스다운코트


...

0.1111111111111111
0.5555555555555556

식품 수산 해산물/어패류 전복
가이거  제니스시계 주얼리세트


0.13333333333333333
0.26666666666666666
0.3076923076923077

패션잡화 주얼리 주얼리세트 패션주얼리세트
비가린 배추김치


0.09090909090909091
0.125
0.14285714285714285

식품 김치 포기김치 
뽕셰프 이봉원 특갈비탕


0.07142857142857142
0.07692307692307693
0.08333333333333333
0.1

식품 냉동/간편조리식품 즉석국/즉석탕 
AAE LA갈비


0.1111111111111111
0.16666666666666666

식품 축산 축산가공식품 쇠고기양념육



In [217]:
cat_df[cat_df['cat1']=='']

,pname,cat1,cat2,cat3,cat4,pname2
214,거창특등급사과,,,,,거창특등급사과
225,아가타 컬러풀에디션,,,,,아가타 컬러풀에디션
244,자미코코 매트릭스,,,,,자미코코 매트릭스
266,메이듀 남성 린넨 블렌디드 슬립온,,,,,블렌디드 슬립온
311,스튜디오럭스 후드롱구스다운코트,,,,,스튜디오럭스 후드롱구스다운코트
314,더커진거창특등급사과,,,,,더커진거창특등급사과
332,한샘 뉴퍼스티지R 리클라이너,,,,,뉴퍼스티지R 리클라이너
353,도냄 쿠쿠전기밥솥,,,,,도냄 쿠쿠전기밥솥
375,바다원 더 그릴 아구채 양태채 쥐포채 소스,,,,,쥐포채 소스
416,삼익가구 LED 제니비 침대,,,,,제니비 침대


In [223]:
full_cat_df = pd.merge(cat_df,unknown,how = 'left', on='pname').drop(['pname2_x','cat1_y','cat2_y','cat3_y','cat4_y','pname2_y'],axis=1)
full_cat_df

,pname,cat1_x,cat2_x,cat3_x,cat4_x
0,프라다 투브릿지 선글라스,패션잡화,선글라스/안경테,선글라스,
1,사운드룩 LP턴테이블 오디오 SLT 7080BT,디지털/가전,음향가전,턴테이블,
2,투홀릭 풍기인견 드로즈,패션의류,남성언더웨어/잠옷,팬티,
3,삼성 블루스카이 공기청정기 AX80R7580WDD,디지털/가전,계절가전,공기정화기,공기청정기
4,PAT 남성 캐주얼 셔츠,패션의류,남성의류,셔츠/남방,
...,...,...,...,...,...
1261,레드캠프 남성 썸머 기능성 셔츠,스포츠/레저,등산,등산의류,반팔티셔츠
1262,유캐슬 마리노 침실가구,가구/인테리어,침실가구,화장대,일반화장대
1263,트레스패스 여성 다이얼락 방한화 기모 워킹화,패션잡화,남성신발,운동화,워킹화
1264,컬럼비아 남성 옴니히트 업 베이스레이어,패션의류,남성의류,트레이닝복,


In [230]:
result_df = pd.merge(p_name,full_cat_df, how='left', left_on='After Process', right_on='pname').drop(['After Process','pname'],axis=1)
result_df

,Before Process,cat1_x,cat2_x,cat3_x,cat4_x
0,테이트 남성 셀린니트3종,패션의류,남성의류,니트/스웨터,
1,테이트 여성 셀린니트3종,패션의류,여성의류,니트/스웨터,
2,오모떼 레이스 파운데이션 브라,패션의류,여성언더웨어/잠옷,브라팬티세트,
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,패션의류,남성의류,바지,
4,보코 리버시블 무스탕,패션의류,여성의류,코트,
...,...,...,...,...,...
1687,조방할매 낙곱새 6팩세트,식품,가공식품,수산가공식품,해물볶음류
1688,파격가 니봇 무선 진공 물걸레 로봇청소기 +뉴헤어드라이어기,디지털/가전,생활가전,청소기,로봇청소기
1689,램프쿡 자동회전냄비,생활/건강,주방용품,냄비/솥,양수냄비
1690,벨라홈 스마트 멀티포트 1+1 세트,디지털/가전,주방가전,전기포트,라면포트


In [236]:
re_performance = pd.merge(performance,result_df, how='left', left_on='상품명',right_on='Before Process').drop(['Before Process'],axis=1)
re_performance

,방송일시,노출(분),상품명,상품군,판매단가,취급액,방송일시_yymmdd,방송일시_hhmmss,seconds,방송일시_hhmm,ratings_mean,cat1_x,cat2_x,cat3_x,cat4_x
0,2019-01-01 06:00:00,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,06:00:00,360,06:00,0.002255,패션의류,남성의류,니트/스웨터,
1,2019-01-01 06:00:00,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,06:00:00,360,06:00,0.002255,패션의류,여성의류,니트/스웨터,
2,2019-01-01 06:20:00,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,06:20:00,380,06:20,0.002210,패션의류,남성의류,니트/스웨터,
3,2019-01-01 06:20:00,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,06:20:00,380,06:20,0.002210,패션의류,여성의류,니트/스웨터,
4,2019-01-01 06:40:00,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,06:40:00,400,06:40,0.001997,패션의류,남성의류,니트/스웨터,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,2019-12-31,23:40:00,1420,23:40,0.004036,디지털/가전,주방가전,전기밥솥,압력밥솥
37368,2020-01-01 00:00:00,20.0,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,2020-01-01,00:00:00,0,00:00,0.004875,디지털/가전,주방가전,전기밥솥,압력밥솥
37369,2020-01-01 00:00:00,20.0,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020-01-01,00:00:00,0,00:00,0.004875,디지털/가전,주방가전,전기밥솥,압력밥솥
37370,2020-01-01 00:00:00,20.0,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020-01-01,00:00:00,0,00:00,0.004875,디지털/가전,주방가전,전기밥솥,압력밥솥


In [241]:
re_performance.to_csv('categorized_performance.csv')